In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import time

In [2]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./AT_interpolated.csv", mode='a', index=False)


In [3]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        

        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,
        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./AT_interpolated.csv", mode='a', index=False, header=False)



In [4]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
#     y_test_tensor = torch.tensor(y_test, dtype=torch.long)
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def2/y_pred_{attack_name}{eps}_Def2.npy", all_preds)

        

In [5]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [7]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [8]:
x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)

train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [10]:
# Adversarial training function with interpolation
def interpolated_adversarial_training(model, train_loader, optimizer, epoch, device, eps=0.3, alpha=2/255, iters=40):
    model.train()
    loss_fn = nn.CrossEntropyLoss()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Create adversarial examples
        data_adv = data.clone().detach().requires_grad_(True).to(device)
        outputs = model(data_adv)
        loss = loss_fn(outputs, target)
        loss.backward()
        data_adv = data_adv + alpha * data_adv.grad.sign()
        eta = torch.clamp(data_adv - data, min=-eps, max=eps)
        data_adv = torch.clamp(data + eta, min=0, max=1).detach_()

        # Interpolation
        lam = np.random.beta(1.0, 1.0)
        data_mixed = lam * data + (1 - lam) * data_adv
        target_mixed = lam * target + (1 - lam) * target

        optimizer.zero_grad()
        output = model(data_mixed)
        target_mixed = target_mixed.long()
        
        loss = loss_fn(output, target_mixed)
        loss.backward()
        optimizer.step()

        if batch_idx % 1000 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}'
                  f' ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# Model initialization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
input_size = input_shape  # Replace with your input shape
output_size = output_shape  # Replace with your output shape
model = DNNModel(input_size=input_size, output_size=output_size).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')



In [11]:
# # Training loop with early stopping and validation evaluation
# for epoch in range(10):
#     interpolated_adversarial_training(model, train_loader, optimizer, epoch, device, eps=0.3, alpha=2/255, iters=40)

#     # Validation
#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break


In [12]:
model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adversarial_Training/Adversarial_Training_interpolated.pt"))


/tmp/ipykernel_172/413485849.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adve

<All keys matched successfully>

In [13]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./AT_interpolated.csv", mode='a', index=False, header=False)

In [13]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
(149, 56) (149,)
Accuracy: 0.9261744966442953

macro
Precision: 0.898380355276907
Recall: 0.8900255754475703
F1 Score: 0.8941009239516702

weighted
Precision: 0.9255047582980932
Recall: 0.9261744966442953
F1 Score: 0.9257833555138973

start XGB 100 BIM
(155, 56) (155,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.5290322580645161

macro
Precision: 0.5
Recall: 0.2645161290322581
F1 Score: 0.3459915611814346

weighted
Precision: 1.0
Recall: 0.5290322580645161
F1 Score: 0.6919831223628692

start XGB 100 FGSM
(80, 56) (80,)
Accuracy: 0.975

macro
Precision: 0.5
Recall: 0.4875
F1 Score: 0.4936708860759494

weighted
Precision: 1.0
Recall: 0.975
F1 Score: 0.9873417721518989

start XGB 100 PGD
(155, 56) (155,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5290322580645161

macro
Precision: 0.5
Recall: 0.2645161290322581
F1 Score: 0.3459915611814346

weighted
Precision: 1.0
Recall: 0.5290322580645161
F1 Score: 0.6919831223628692

start XGB 100 DF
(687, 56) (687,)
Accuracy: 0.9446870451237264

macro
Precision: 0.49693721286370596
Recall: 0.4751098096632504
F1 Score: 0.48577844311377244

weighted
Precision: 0.9880876750681548
Recall: 0.9446870451237264
F1 Score: 0.9659000775741093

start XGB 100 AutoPGD
(365, 56) (365,)
Accuracy: 0.8136986301369863

macro
Precision: 0.7753488372093024
Recall: 0.8700925231307827
F1 Score: 0.7871062924586565

weighted
Precision: 0.8892182223638102
Recall: 0.8136986301369863
F1 Score: 0.8268917278998786

start XGB 100 ZOO
(436, 56) (436,)
Accuracy: 0.9151376146788991

macro
Precision: 0.8807399632642351
Recall: 0.8833215921098978
F1 Score: 0.8820199656269426

weighted
Precision: 0.9154367246912005
Recall: 0.9151376146788991
F1 Score: 0.9152809811250114

start XGB 100 CaFA
(102, 56) (102,)
Accuracy

...

Accuracy: 0.37209302325581395

macro
Precision: 0.5344827586206896
Recall: 0.6707317073170732
F1 Score: 0.3190615835777126

weighted
Precision: 0.9566960705693666
Recall: 0.37209302325581395
F1 Score: 0.49141376253154195

start XGB 1 DF
(49, 56) (49,)
Accuracy: 0.9387755102040817

macro
Precision: 0.5
Recall: 0.46938775510204084
F1 Score: 0.4842105263157895

weighted
Precision: 1.0
Recall: 0.9387755102040817
F1 Score: 0.968421052631579

start XGB 1 AutoPGD
(79, 56) (79,)
Accuracy: 0.4810126582278481

macro
Precision: 0.4813793103448276
Recall: 0.4826923076923077
F1 Score: 0.4725614720729523

weighted
Precision: 0.48139676996944564
Recall: 0.4810126582278481
F1 Score: 0.4717163534574627

start XGB 1 ZOO
(80, 56) (80,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.7

macro
Precision: 0.7047619047619047
Recall: 0.701688555347092
F1 Score: 0.6992481203007519

weighted
Precision: 0.7057142857142857
Recall: 0.7
F1 Score: 0.6988721804511279

start XGB 1 CaFA
(5, 56) (5,)
Accuracy: 0.4

macro
Precision: 0.625
Recall: 0.625
F1 Score: 0.4

weighted
Precision: 0.85
Recall: 0.4
F1 Score: 0.4

start XGB 1 SINIFGSM
(31, 56) (31,)
Accuracy: 0.6129032258064516

macro
Precision: 0.6666666666666666
Recall: 0.76
F1 Score: 0.5921052631578947

weighted
Precision: 0.8709677419354839
Recall: 0.6129032258064516
F1 Score: 0.6485568760611206

start XGB 1 VNIFGSM
(12, 56) (12,)
Accuracy: 0.9166666666666666

macro
Precision: 0.8333333333333333
Recall: 0.95
F1 Score: 0.8736842105263158

weighted
Precision: 0.9444444444444445
Recall: 0.9166666666666666
F1 Score: 0.9228070175438595

start RF 100 baseline
(229, 56) (229,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 100 BIM
(2, 56) (2,)

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.9259259259259259

macro
Precision: 0.9308270676691729
Recall: 0.9134199134199135
F1 Score: 0.9205882352941177

weighted
Precision: 0.9271512113617377
Recall: 0.9259259259259259
F1 Score: 0.9251633986928105

start RF 1 CaFA
(1, 56) (1,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input1/x_test_adv_SINIFGSM_Def2.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF/UNSW_Input1/x_test_adv_VNIFGSM_Def2.npy not found
start DT 100 baseline
(429, 56) (429,)
Accuracy: 0.9393939393939394

macro
Precision: 0.9572541291291292
Recall: 0.8942152103559871
F1 Score: 0.9195655936310142

weighted
Precision: 0.943011193011193
Recall: 0.9393939393939394
F1 Score: 0.9371597595896661

start DT 100 BIM
(3358, 56) (3358,)
Accuracy: 0.3412745681953544

macro
Precision: 0.4585025336

In [12]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()



start XGB 100 baseline
(137, 56) (137,)
Accuracy: 0.8832116788321168

macro
Precision: 0.7887500000000001
Recall: 0.8083524027459954
F1 Score: 0.7979351032448377

weighted
Precision: 0.8875547445255474
Recall: 0.8832116788321168
F1 Score: 0.8851280063734039

start XGB 100 BIM
(59, 56) (59,)
Accuracy: 0.8813559322033898

macro
Precision: 0.5
Recall: 0.4406779661016949
F1 Score: 0.46846846846846846

weighted
Precision: 1.0
Recall: 0.8813559322033898
F1 Score: 0.9369369369369369

start XGB 100 FGSM
(56, 56) (56,)
Accuracy: 0.9642857142857143

macro
Precision: 0.6666666666666666
Recall: 0.9818181818181818
F1 Score: 0.7407407407407407

weighted
Precision: 0.9880952380952381
Recall: 0.9642857142857143
F1 Score: 0.9728835978835979

start XGB 100 PGD
(59, 56) (59,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.p

Accuracy: 0.8813559322033898

macro
Precision: 0.5
Recall: 0.4406779661016949
F1 Score: 0.46846846846846846

weighted
Precision: 1.0
Recall: 0.8813559322033898
F1 Score: 0.9369369369369369

start XGB 100 DF
(680, 56) (680,)
Accuracy: 0.9529411764705882

macro
Precision: 0.5148533950617284
Recall: 0.7271386430678466
F1 Score: 0.5173453996983408

weighted
Precision: 0.9956120642701525
Recall: 0.9529411764705882
F1 Score: 0.973170082512643

start XGB 100 AutoPGD
(278, 56) (278,)
Accuracy: 0.8848920863309353

macro
Precision: 0.8
Recall: 0.9304347826086956
F1 Score: 0.8376168224299065

weighted
Precision: 0.9309352517985612
Recall: 0.8848920863309353
F1 Score: 0.8949774759631547

start XGB 100 ZOO
(392, 56) (392,)
Accuracy: 0.8775510204081632

macro
Precision: 0.7868769716088329
Recall: 0.8060304213218468
F1 Score: 0.7958333333333334

weighted
Precision: 0.8820472542329235
Recall: 0.8775510204081632
F1 Score: 0.8795280612244899

start XGB 100 CaFA
(18, 56) (18,)
Accuracy: 0.611111111111111

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.19533527696793002

macro
Precision: 0.4785714285714286
Recall: 0.09852941176470588
F1 Score: 0.16341463414634147

weighted
Precision: 0.948771345272803
Recall: 0.19533527696793002
F1 Score: 0.32397070326388394

start XGB 100 VNIFGSM
(64, 56) (64,)
Accuracy: 0.859375

macro
Precision: 0.5
Recall: 0.4296875
F1 Score: 0.46218487394957986

weighted
Precision: 1.0
Recall: 0.859375
F1 Score: 0.9243697478991597

start XGB 50 baseline
(155, 56) (155,)


...

Accuracy: 0.9938775510204082

macro
Precision: 0.7479508196721312
Recall: 0.6656399726214921
F1 Score: 0.6984615384615385

weighted
Precision: 0.9928655068584811
Recall: 0.9938775510204082
F1 Score: 0.9932684458398744

start RF 1 AutoPGD
(21, 56) (21,)
Accuracy: 0.7142857142857143

macro
Precision: 0.35714285714285715
Recall: 0.5
F1 Score: 0.4166666666666667

weighted
Precision: 0.5102040816326531
Recall: 0.7142857142857143
F1 Score: 0.5952380952380952

start RF 1 ZOO
(24, 56) (24,)
Accuracy: 0.625

macro
Precision: 0.3125
Recall: 0.5
F1 Score: 0.38461538461538464

weighted
Precision: 0.390625
Recall: 0.625
F1 Score: 0.4807692307692308

start RF 1 CaFA
(1, 56) (1,)
Accuracy: 0.0

macro
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

weighted
Precision: 0.0
Recall: 0.0
F1 Score: 0.0

start RF 1 SINIFGSM
/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_ExcludeCaFA/UNSW_Input1/x_test_adv_SINIFGSM_Def2.npy not found
start RF 1 VNIFGSM
/home/jovyan/Sample_Based_Extension/U

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

Accuracy: 0.9549763033175356

macro
Precision: 0.9672680328136746
Recall: 0.9165792540792541
F1 Score: 0.9383813776196368

weighted
Precision: 0.9567471457025725
Recall: 0.9549763033175356
F1 Score: 0.9536881127382465

start DT 100 BIM
(928, 56) (928,)
Accuracy: 0.6648706896551724

macro
Precision: 0.6608347738805553
Recall: 0.6881370028320296
F1 Score: 0.6506865646351132

weighted
Precision: 0.7322704830912765
Recall: 0.6648706896551724
F1 Score: 0.6775377959565089

start DT 100 FGSM
(1081, 56) (1081,)
Accuracy: 0.393154486586494

macro
Precision: 0.4868146323130246
Recall: 0.4864111723188156
F1 Score: 0.3930376193149916

weighted
Precision: 0.586596356191616
Recall: 0.393154486586494
F1 Score: 0.39684749236597017

start DT 100 PGD
(928, 56) (928,)
Accuracy: 0.6648706896551724

macro
Precision: 0.6608347738805553
Recall: 0.6881370028320296
F1 Score: 0.6506865646351132

weighted
Precision: 0.7322704830912765
Recall: 0.6648706896551724
F1 Score: 0.6775377959565089

start DT 100 DF
(803,

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.6801801801801802

macro
Precision: 0.5
Recall: 0.3400900900900901
F1 Score: 0.40482573726541554

weighted
Precision: 1.0
Recall: 0.6801801801801802
F1 Score: 0.8096514745308311

start DT 20 SINIFGSM
(4893, 56) (4893,)
Accuracy: 0.8029838544860004

macro
Precision: 0.5843520585946751
Recall: 0.6345519768386232
F1 Score: 0.5963136641114754

weighted
Precision: 0.8559091307248228
Recall: 0.8029838544860004
F1 Score: 0.8252978596164118

start DT 20 VNIFGSM
(2336, 56) (2336,)
Accuracy: 0.8159246575342466

macro
Precision: 0.6674727958072755
Recall: 0.7378582256933559
F1 Score: 0.6890955642385614

weighted
Precision: 0.8582095676126794
Recall: 0.8159246575342466
F1 Score: 0.831735791148856

start DT 1 baseline
(131, 56) (131,)
Accuracy: 0.7251908396946565

macro
Precision: 0.6623931623931625
Recall: 0.6270903010033445
F1 Score: 0.6354359925788498

weighted
Precision: 0.7056175376786065
Recall: 0.7251908396946565
F1 Score: 0.7086207140732768

start DT 1 BIM
(2280, 56) (2280,)
Accu

In [11]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

percentage = ["100", "50", "20", "1"]
model_name = ["XGB", "RF", "DT" ]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



start XGB 100 baseline
(240, 56) (240,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

Accuracy: 0.7125

macro
Precision: 0.35625
Recall: 0.5
F1 Score: 0.41605839416058393

weighted
Precision: 0.50765625
Recall: 0.7125
F1 Score: 0.5928832116788321

start XGB 100 BIM
(145, 56) (145,)
Accuracy: 0.9793103448275862

macro
Precision: 0.4896551724137931
Recall: 0.5
F1 Score: 0.49477351916376305

weighted
Precision: 0.9590487514863257
Recall: 0.9793103448275862
F1 Score: 0.9690736513276463

start XGB 100 FGSM
(80, 56) (80,)
Accuracy: 1.0

macro
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

weighted
Precision: 1.0
Recall: 1.0
F1 Score: 1.0

start XGB 100 PGD
(145, 56) (145,)
Accuracy: 0.9793103448275862

macro
Precision: 0.4896551724137931
Recall: 0.5
F1 Score: 0.49477351916376305

weighted
Precision: 0.9590487514863257
Recall: 0.9793103448275862
F1 Score: 0.9690736513276463

start XGB 100 DF
(689, 56) (689,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

Accuracy: 0.9956458635703919

macro
Precision: 0.49782293178519593
Recall: 0.5
F1 Score: 0.4989090909090909

weighted
Precision: 0.9913106856448315
Recall: 0.9956458635703919
F1 Score: 0.9934735453226019

start XGB 100 AutoPGD
(603, 56) (603,)


...

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.5552941176470588

macro
Precision: 0.2776470588235294
Recall: 0.5
F1 Score: 0.35703479576399394

weighted
Precision: 0.3083515570934256
Recall: 0.5552941176470588
F1 Score: 0.3965186437661297

start DT 1 CaFA
(6672, 56) (6672,)
Accuracy: 0.19934052757793766

macro
Precision: 0.09967026378896883
Recall: 0.5
F1 Score: 0.166208447888028

weighted
Precision: 0.039736645935050524
Recall: 0.19934052757793766
F1 Score: 0.06626415937981932

start DT 1 SINIFGSM
(3401, 56) (3401,)


/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/m

Accuracy: 0.3678329902969715

macro
Precision: 0.18391649514848574
Recall: 0.5
F1 Score: 0.26891659501289766

weighted
Precision: 0.13530110875081192
Recall: 0.3678329902969715
F1 Score: 0.1978327905681476

start DT 1 VNIFGSM
(686, 56) (686,)
Accuracy: 0.5116618075801749

macro
Precision: 0.25583090379008744
Recall: 0.5
F1 Score: 0.33847637415621984

weighted
Precision: 0.26179780533621194
Recall: 0.5116618075801749
F1 Score: 0.3463708668479101



/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
epsilon = 0

Def = "Def2"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9711538461538461

macro
Precision: 0.9730620878437228
Recall: 0.969924192485168
F1 Score: 0.970996229923049

weighted
Precision: 0.9720443589424551
Recall: 0.9711538461538461
F1 Score: 0.9711058759966471

start BIM
Accuracy: 0.6095505617977528

macro
Precision: 0.5943458980044346
Recall: 0.5824745598449362
F1 Score: 0.5796144795303753

weighted
Precision: 0.6003830439223699
Recall: 0.6095505617977528
F1 Score: 0.5966307789244635

start FGSM
Accuracy: 0.6520547945205479

macro
Precision: 0.6462015503875969
Recall: 0.6435574229691876
F1 Score: 0.6442686230421537

weighted
Precision: 0.6500736965063183
Recall: 0.6520547945205479
F1 Score: 0.6504687225527269

start PGD
Accuracy: 0.6095505617977528

macro
Precision: 0.5943458980044346
Recall: 0.5824745598449362
F1 Score: 0.5796144795303753

weighted
Precision: 0.6003830439223699
Recall: 0.6095505617977528
F1 Score: 0.5966307789244635

start DF
Accuracy: 0.8621190130624092

macro
Precision: 0.5152391975308642
Recal

In [18]:
# torch.save(model.state_dict(), "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/Adversarial_Training/Adversarial_Training_interpolated.pt")

In [ ]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")